In [4]:
from tweepy import API 
from tweepy import Cursor
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
 
import numpy as np
import pandas as pd

 
ACCESS_TOKEN = "545740744-KCNbGN3BE31JXNUw2KA66t3j0HzetLQnvABBKYwO"
ACCESS_TOKEN_SECRET = "LtRU7TImpTcO0aa26lpsmlgGgPZmXZB3XyA1zHtZ2zoHO"
CONSUMER_KEY = "DWKn6LX6Kpv96nrMnM2DGz05U"
CONSUMER_SECRET = "948K7MubgVcKbTCZ22cO2a434NZxPvBTPMjUkUeHco89QqTbib"
 

# # # # TWITTER CLIENT # # # #
class TwitterClient():
    def __init__(self, twitter_user=None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth)

        self.twitter_user = twitter_user

    def get_twitter_client_api(self):
        return self.twitter_client

    def get_user_timeline_tweets(self, num_tweets):
        tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline, id=self.twitter_user).items(num_tweets):
            tweets.append(tweet)
        return tweets

    def get_friend_list(self, num_friends):
        friend_list = []
        for friend in Cursor(self.twitter_client.friends, id=self.twitter_user).items(num_friends):
            friend_list.append(friend)
        return friend_list

    def get_home_timeline_tweets(self, num_tweets):
        home_timeline_tweets = []
        for tweet in Cursor(self.twitter_client.home_timeline, id=self.twitter_user).items(num_tweets):
            home_timeline_tweets.append(tweet)
        return home_timeline_tweets


# # # # TWITTER AUTHENTICATER # # # #
class TwitterAuthenticator():

    def authenticate_twitter_app(self):
        auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
        auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
        return auth

# # # # TWITTER STREAMER # # # #
class TwitterStreamer():
    """
    Class for streaming and processing live tweets.
    """
    def __init__(self):
        self.twitter_autenticator = TwitterAuthenticator()    

    def stream_tweets(self, fetched_tweets_filename, hash_tag_list):
        # This handles Twitter authetification and the connection to Twitter Streaming API
        listener = TwitterListener(fetched_tweets_filename)
        auth = self.twitter_autenticator.authenticate_twitter_app() 
        stream = Stream(auth, listener)

        # This line filter Twitter Streams to capture data by the keywords: 
        stream.filter(track=hash_tag_list)


# # # # TWITTER STREAM LISTENER # # # #
class TwitterListener(StreamListener):
    """
    This is a basic listener that just prints received tweets to stdout.
    """
    def __init__(self, fetched_tweets_filename):
        self.fetched_tweets_filename = fetched_tweets_filename

    def on_data(self, data):
        try:
            print(data)
            with open(self.fetched_tweets_filename, 'a') as tf:
                tf.write(data)
            return True
        except BaseException as e:
            print("Error on_data %s" % str(e))
        return True
          
    def on_error(self, status):
        if status == 420:
            # Returning False on_data method in case rate limit occurs.
            return False
        print(status)


class TweetAnalyzer():
    """
    Functionality for analyzing and categorizing content from tweets.
    """
    def tweets_to_data_frame(self, tweets):
        df = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])

        df['id'] = np.array([tweet.id for tweet in tweets])
        df['len'] = np.array([len(tweet.text) for tweet in tweets])
        df['date'] = np.array([tweet.created_at for tweet in tweets])
        df['source'] = np.array([tweet.source for tweet in tweets])
        df['likes'] = np.array([tweet.favorite_count for tweet in tweets])
        df['retweets'] = np.array([tweet.retweet_count for tweet in tweets])

        return df

 
if __name__ == '__main__':

    twitter_client = TwitterClient()
    tweet_analyzer = TweetAnalyzer()

    api = twitter_client.get_twitter_client_api()

    tweets = api.user_timeline(screen_name="jokowi", count=20)

    #print(dir(tweets[0]))
    #print(tweets[0].retweet_count)

    df = tweet_analyzer.tweets_to_data_frame(tweets)
    
    print(df)


                                               Tweets                   id  \
0   Upacara di Istana Merdeka hari ini berlangsung...  1162684138127679488   
1               Cilukbaaa ... https://t.co/yVI0LBAY5X  1162627082322468864   
2   Berbagi semangat kemerdekaan dengan anak cucu,...  1162576540455952384   
3   Hari ini, di Hari Ulang Tahun Republik Indones...  1162554455608324102   
4   Pukul 00.00 WIB tadi, di Taman Makam Pahlawan,...  1162533101941055488   
5   Rumah besar Indonesia adalah tempat yang nyama...  1162308540708470786   
6   Bersama menuju Indonesia maju... https://t.co/...  1162285055353376770   
7   Dengan baju adat Sasak, NTB, saya menyampaikan...  1162257445055369217   
8   Tegap-tegap dan wajah bersemangat. Mereka, 68 ...  1162244503995949058   
9   Dunia bergerak cepat, diwarnai kejutan yang ta...  1162205842743033858   
10  Sebanyak 29 anak bangsa hari ini menerima anug...  1161947530160046080   
11  Selamat ulang tahun ke-58 Gerakan Pramuka.\n\n...  116163675